In [ ]:
import pandas as pd
import json
import ast

In [ ]:
def convert_to_dict_non_diff(dataframe, dictionary, set_evidences, set_conditions):
    for index, row in dataframe.iterrows():
        dict_record = {}
        dict_record['AGE'] = row['AGE']
        dict_record['SEX'] = row['SEX']
        evidences_string = row['EVIDENCES']
        evidences_parts = ast.literal_eval(evidences_string)
        differential_diagnosis_string = row['PATHOLOGY']
        for evidence in evidences_parts:
            dict_record[evidence] = 1
        for evidence in set_evidences:
            if evidence not in dict_record:
                dict_record[evidence] = 0
        dict_record[differential_diagnosis_string] = 1
        for diagnosis in set_conditions:
            if diagnosis not in dict_record:
                dict_record[diagnosis] = 0
        for key, val in dict_record.items():
            dictionary[key].append(val)

In [80]:
def introduce_conditions(conditions, dict_for_insert):
    for condition in conditions:
        dict_for_insert[condition] = []

In [81]:
def introduce_evidences(evidences, dict_for_insert):
    for evidence in evidences:
        dict_for_insert[evidence] = []

In [ ]:
with open("huggingface_dataset/ddxplus/release_evidences.json", "r") as f:
    release_evidences = json.load(f)

with open("huggingface_dataset/ddxplus/release_conditions.json", "r") as f:
    release_conditions = json.load(f)

In [ ]:
set_conditions = [a for a in release_conditions.keys()]
set_evidences = []
for key, value in release_evidences.items():
    if len(value['possible-values']) > 0:
        for val in value['possible-values']:
            set_evidences.append(f'{key}_@_{val}')
    else:
        set_evidences.append(f'{key}')

In [ ]:
dict_data = {}
dict_data['AGE'] = []
dict_data['SEX'] = []
introduce_evidences(set_evidences, dict_data)
introduce_conditions(set_conditions, dict_data)

In [ ]:
df = pd.read_csv("huggingface_dataset/ddxplus/train.csv")
df['AGE'] = df['AGE'].fillna(df['AGE'].median())
df['SEX'] = df['SEX'].fillna(df['SEX'].mode()[0])
convert_to_dict_non_diff(df, dict_data, set_evidences, set_conditions)

In [ ]:
df_preproccessed = pd.DataFrame(dict_data)
df_preproccessed.to_csv("preprocessed_dataset/preprocessed_train_non_diff.csv")

In [ ]:
dict_data_test = {}
dict_data_test['AGE'] = []
dict_data_test['SEX'] = []
introduce_evidences(set_evidences, dict_data_test)
introduce_conditions(set_conditions, dict_data_test)

In [ ]:
df_test = pd.read_csv("huggingface_dataset/ddxplus/test.csv")
df_test['AGE'] = df_test['AGE'].fillna(df_test['AGE'].median())
df_test['SEX'] = df_test['SEX'].fillna(df_test['SEX'].mode()[0])
convert_to_dict_non_diff(df_test, dict_data_test, set_evidences, set_conditions)

In [ ]:
df_test_preproccessed = pd.DataFrame(dict_data_test)
df_test_preproccessed.to_csv("preprocessed_dataset/preprocessed_test_nondiff.csv")

In [ ]:
dict_data_validate = {}
dict_data_validate['AGE'] = []
dict_data_validate['SEX'] = []
introduce_evidences(set_evidences, dict_data_validate)
introduce_conditions(set_conditions, dict_data_validate)

In [ ]:
df_validate = pd.read_csv("huggingface_dataset/ddxplus/validate.csv")
df_validate['AGE'] = df_validate['AGE'].fillna(df_validate['AGE'].median())
df_validate['SEX'] = df_validate['SEX'].fillna(df_validate['SEX'].mode()[0])
convert_to_dict_non_diff(df_validate, dict_data_validate, set_evidences, set_conditions)

In [ ]:
df_validate_preproccessed = pd.DataFrame(dict_data_validate)
df_validate_preproccessed.to_csv("preproccessed_dataset/preprocessed_validate_non_diff.csv")